# Initial Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS

Mounted at /content/drive
/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS


In [2]:
project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"
!ln -s {project_path} /content/cad_detection

## Building the environment

In [3]:
# ==============================================================================
# The Final, Corrected, and Complete Installation Script
# ==============================================================================
import os

# --- Part 1: Setup Python 3.8 with Development Headers ---
print("--- [Part 1/4] Setting up Python 3.8 (with -dev package) ---")
# This line adds python3.8-dev, which provides Python.h and fixes the final compile error.
!sudo apt-get update -y > /dev/null
!sudo apt-get install -y python3.8 python3.8-distutils python3.8-dev > /dev/null
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1 > /dev/null
!curl -s https://bootstrap.pypa.io/pip/3.8/get-pip.py -o get-pip.py
!python3.8 get-pip.py > /dev/null
print("Python 3.8 and development headers are set up.")


# --- Part 2: Install a Compatible Modern PyTorch ---
print("\n--- [Part 2/4] Installing a modern, compatible PyTorch ---")
!python3.8 -m pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu121
print("PyTorch is installed.")


# --- Part 3: Install Remaining Requirements ---
print("\n--- [Part 3/4] Installing remaining packages from requirements.txt ---")
project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"
req_file = os.path.join(project_path, "requirements.txt")

# Ensure your fixes for plyfile==1.0.3 and scipy==1.10.1 are in the file.
# Then, comment out ALL torch-related lines to prevent conflicts.
!sed -i '/^torch/s/^/#/' {req_file}
!python3.8 -m pip install --ignore-installed -r {req_file}
print("Remaining packages are installed.")


# --- Part 4: The Final Manual Build ---
print("\n--- [Part 4/4] Manually compiling custom modules ---")
submodules_path = os.path.join(project_path, "submodules")

# Install ninja for faster compilation
!python3.8 -m pip install ninja > /dev/null

# Download and prepare simple-knn from the CORRECT GitLab source
os.chdir(submodules_path)
if not os.path.exists('simple-knn'):
    print("Downloading simple-knn from GitLab...")
    !wget -q https://gitlab.inria.fr/bkerbl/simple-knn/-/archive/main/simple-knn-main.zip
    !unzip -o -q simple-knn-main.zip
    !mv simple-knn-main simple-knn
    !rm simple-knn-main.zip
    print("Successfully downloaded and extracted simple-knn.")
else:
    print("'simple-knn' directory already exists.")
os.chdir(project_path)

# Define the build command with the CUDA Arch fix
build_command = 'TORCH_CUDA_ARCH_LIST="7.5 8.0 8.6" python3.8 setup.py install'

# Build diff-surfel-rasterization
print("Building 'diff-surfel-rasterization'...")
rasterizer_path = os.path.join(submodules_path, "diff-surfel-rasterization")
os.chdir(rasterizer_path)
!{build_command}
os.chdir(project_path)

# Build simple-knn
print("Building 'simple-knn'...")
knn_path = os.path.join(submodules_path, "simple-knn")
os.chdir(knn_path)
!{build_command}
os.chdir(project_path)

print("\n\n✅✅✅ If there are no errors above, the setup is finally complete. ✅✅✅")

--- [Part 1/4] Setting up Python 3.8 (with -dev package) ---
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Python 3.8 and development headers are set up.

--- [Part 2/4] Installing a modern, compatible PyTorch ---
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 

## Github Access

### Setup

In [6]:
# ==============================================================================
# CELL 1: GIT SETUP (RUN ONCE PER SESSION)
# ==============================================================================
import os
from google.colab import userdata

# --- 1. Configuration (Fill this out once) ---
# The full path to your project folder in Google Drive
project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"

# Your GitHub details
github_username = "alirezafalah"
your_name = "Alireza Falah"
your_email = "al.r.falah@gmail.com"
repo_name = "CAD_Sparse2DGS"

# --- 2. Execution ---
print(f"Navigating to: {project_path}")
os.chdir(project_path)

print("\nConfiguring Git user for this session...")
!git config --global user.name "{your_name}"
!git config --global user.email "{your_email}"

print("Preparing authenticated connection to GitHub...")
try:
    # Get the token from Colab's secret manager
    github_token = userdata.get('GITHUB_TOKEN')
    # Construct the authenticated remote URL and store it for this session
    remote_url = f"https://{github_token}@github.com/{github_username}/{repo_name}.git"
    print("\n✅ Setup complete. You can now use the 'Commit & Push' cell.")
except userdata.SecretNotFoundError:
    print("\n\n❌ ERROR: Secret 'GITHUB_TOKEN' not found. Please add it to the secret manager (🔑 icon).")
except Exception as e:
    print(f"\n\n❌ An error occurred during setup: {e}")

Navigating to: /content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS

Configuring Git user for this session...
Preparing authenticated connection to GitHub...

✅ Setup complete. You can now use the 'Commit & Push' cell.


### Commit

In [ ]:
# ==============================================================================
# CELL 2: COMMIT & PUSH (RUN AS NEEDED)
# ==============================================================================

#@markdown ### Fill out the details for this commit:
#@markdown Type the filenames you want to commit, separated by spaces.
files_to_commit = "2DGS_test.ipynb requirements.txt" #@param {type:"string"}
#@markdown Type a descriptive message for your commit.
commit_message = "Envionment Setup" #@param {type:"string"}

# --- Execution ---
print("--- Checking Git Status ---")
!git status

print(f"\n--- Staging files: {files_to_commit} ---")
!git add {files_to_commit}

print(f"\n--- Committing with message: '{commit_message}' ---")
# The '-m' flag takes the commit message.
# We wrap the variable in quotes to handle spaces.
!git commit -m "{commit_message}"

print("\n--- Pushing changes to GitHub ---")
# The 'remote_url' variable was created in the setup cell.
# We are pushing to the 'main' branch. Change if your branch is named 'master'.
!git push {remote_url} main

print("\n\n✅ Push complete!")

--- Checking Git Status ---
